In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np

# Обучение без учителя (Unsupervised Learning)


Во многих случаях обучения без учителя, такого как уменьшение размерности, обучение многообразию и извлечение признаков, находят новое представление входных данных без каких-либо дополнительных входных данных.

<img src="figures/unsupervised_workflow.svg" width="100%">

Самый простой пример этого, который едва ли можно назвать обучением, — это изменение масштаба данных для получения нулевого среднего значения и единичной дисперсии. Это полезный шаг предварительной обработки для многих моделей машинного обучения.

Применение такой предварительной обработки имеет интерфейс, очень похожий на алгоритмы контролируемого обучения, которые мы видели до сих пор.

Давайте загрузим набор данных iris и изменим его масштаб:

In [ ]:
from sklearn.datasets import load_iris

iris = load_iris()
X, y = iris.data, iris.target
print(X.shape)

Набор данных iris не «центрирован», то есть имеет ненулевое среднее значение, а стандартное отклонение различно для каждого компонента:

In [ ]:
print("mean : %s " % X.mean(axis=0))
print("standard deviation : %s " % X.std(axis=0))

Чтобы использовать какой-либо метод предварительной обработки, мы сначала импортируем оценщик, здесь `StandardScaler`, и создаем его экземпляр:

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

Как и в случае с алгоритмами классификации и регрессии, вызываем «подгонку», чтобы изучить модель на основе данных. Поскольку это неконтролируемая модель, мы передаем только ``X``, без ``y``. 

Это просто оценивает среднее значение и стандартное отклонение.

In [ ]:
scaler.fit(X)

Теперь можем масштабировать наши данные, применив метод `transform` вместо `predict`:

In [ ]:
X_scaled = scaler.transform(X)

`X_scaled` имеет тоже количество образцов и признаков, но из каждого признака вычли среднее значение, и все признаки отмасштабированы для получения единичного стандартного отклонения:

In [ ]:
print(X_scaled.shape)

In [ ]:
print("mean : %s " % X_scaled.mean(axis=0))
print("standard deviation : %s " % X_scaled.std(axis=0))

Метод главных компонент (Principal Component Analysis)
============================

Более интересное преобразование — это анализ главных компонент (PCA).
Это метод уменьшения размерности данных путем создания линейной проекции.
То есть мы находим новые признаки для представления данных, которые представляют собой линейную комбинацию старых данных (т. е. мы их вращаем).

PCA находит эти новый базис путем поиска базиса с максимальной дисперсией.
Обычно сохраняются лишь несколько компонентов, которые объясняют большую часть различий в данных. Чтобы проиллюстрировать, как может выглядеть вращение, мы сначала покажем его на двумерных данных и сохраним оба главных компонента.

Мы создаем набор объектов с нормальным распределением признаков:

In [ ]:
rnd = np.random.RandomState(5)
X_ = rnd.normal(size=(300, 2))
X_blob = np.dot(X_, rnd.normal(size=(2, 2))) + rnd.normal(size=2)
y = X_[:, 0] > 0
plt.scatter(X_blob[:, 0], X_blob[:, 1], c=y, linewidths=0, s=30)
plt.xlabel("feature 1")
plt.ylabel("feature 2")
plt.show()

Как всегда, мы создаем экземпляр нашей модели PCA. По умолчанию все направления сохраняются.

In [ ]:
from sklearn.decomposition import PCA
pca = PCA()

Затем мы подгоняем модель PCA к нашим данным. Поскольку PCA является неконтролируемым алгоритмом, выходной сигнал `y` отсутствует.

In [ ]:
pca.fit(X_blob)

Затем преобразовываем данные, спроецированные на главные компоненты:

In [ ]:
X_pca = pca.transform(X_blob)

plt.scatter(X_pca[:, 0], X_pca[:, 1], c=y, linewidths=0, s=30)
plt.xlabel("first principal component")
plt.ylabel("second principal component")
plt.show()

Слева на графике вы можете увидеть четыре точки, которые раньше были сверху справа. PCA счел целесообразным, чтобы первый компонент располагался по диагонали, а второй - перпендикулярно ей. Поскольку PCA находит такое вращение, чтобы основные компоненты всегда находилисЬ под прямым углом друг к другу.

Снижение размерности для визуализации с помощью PCA
--------------------------------------------------------------
Рассмотрим набор данных digits. Его нельзя визуализировать на одном двумерном графике, так как он содержит 64 признака. Мы собираемся извлечь 2 измерения для его визуализации, используя пример из sklearn-примеров [ссылка здесь](http://scikit-learn.org/stable/auto_examples/manifold/plot_lle_digits.html)

In [ ]:
from figures.plot_digits_datasets import digits_plot

digits_plot()

Обратите внимание, что эта проекция была определена *без* какой-либо информации о метках (представленных цветами): в этом смысле это **обучение без учителя**, то есть является **неконтролируемым**. Тем не менее, мы видим, что проекция дает нам представление о распределении различных цифр в пространстве параметров.

## Manifold Learning (Обучение многообразиям)

Одной из слабых сторон PCA является то, что он не может обнаружить нелинейные признаки. Набор
алгоритмов, известных как *Manifold Learning*, был разработан для устранения
этого недостатка. Канонический набор данных, используемый в Manifold learning, — это
*S-кривая*, которую мы кратко рассмотрели в предыдущем разделе:

In [ ]:
from sklearn.datasets import make_s_curve
X, y = make_s_curve(n_samples=1000)

from mpl_toolkits.mplot3d import Axes3D
ax = plt.axes(projection='3d')

ax.scatter3D(X[:, 0], X[:, 1], X[:, 2], c=y)
ax.view_init(10, -60)
plt.show()

Это двумерный набор данных, встроенный в три измерения, но он встроен таким образом, что PCA не может обнаружить базовую ориентацию данных:

In [ ]:
from sklearn.decomposition import PCA

X_pca = PCA(n_components=2).fit_transform(X)
plt.scatter(X_pca[:, 0], X_pca[:, 1], c=y)
plt.show()

Однако алгоритмы обучения многообразиям, доступные в подмодуле ``sklearn.manifold``, способны восстановить базовое двумерное многообразие:

In [ ]:
from sklearn.manifold import Isomap

iso = Isomap(n_neighbors=15, n_components=2)
X_iso = iso.fit_transform(X)
plt.scatter(X_iso[:, 0], X_iso[:, 1], c=y)
plt.show()

## Упражнение
Сравните результаты Isomap и PCA на 5-классовом подмножестве набора данных цифр (``load_digits(5)``).

__Бонус__: Также сравните с t-SNE, еще одним популярным методом обучения многообразиям.

In [ ]:
from sklearn.datasets import load_digits

digits = load_digits(n_class=5)

# ...

In [ ]:
from sklearn.decomposition import PCA


In [ ]:
from sklearn.manifold import Isomap


In [18]:
from sklearn.manifold import TSNE